In [2]:
## imports classiques
import numpy as np
import matplotlib.pyplot as plt
import math 
import pandas as pd
import plotly.graph_objects as go
import matplotlib.colors as colors
import random
colors_list = list(colors._colors_full_map.values())
random.shuffle(colors_list)

%matplotlib notebook

In [3]:
# Importation du solveur
from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPArray

In [4]:
# Importation des donees
sites = pd.read_csv('Sites.csv')

onshore = sites[sites['capacite offshore']=='Non'].copy()
offshore = sites[sites['capacite offshore']=='Oui'].copy()

In [6]:
# Modification pour faire les calculs sur un mois
N_heures_par_annee = 24*365 
N_annees = 1/12 # 1/12 ou lieu de 1
N_heures = int(np.ceil(N_heures_par_annee*N_annees))

In [7]:
# importation des donnees
matrice_rendements_offshore = np.genfromtxt('Rendements_offshore.csv', delimiter=',')[:,:N_heures]
matrice_rendements_onshore = np.genfromtxt('Rendements_onshore.csv', delimiter=',')[:,:N_heures]

In [1]:
# Puissance des chaque site
pui = sites['capacites']
pui = np.array(pui)

# creation de matrice rendements
# 1) cap_of
cap_off = sites['capacite offshore']
cap_off = np.where(cap_off == 'Oui',1,cap_off)
cap_off = np.where(cap_off == 'Non',0,cap_off)

# 2) cap_on
cap_on = sites['capacite offshore']
cap_on = np.where(cap_on == 'Oui',0,cap_on)
cap_on = np.where(cap_on == 'Non',1,cap_on)

# 3) multiplication
mat_off = matrice_rendements_offshore.transpose() * cap_off
mat_off = mat_off.transpose()

mat_on = matrice_rendements_onshore.transpose() * cap_on
mat_on = mat_on.transpose()

# 4) addition
mat_tot = mat_on + mat_off

# matrice de production potentielle d'energie
prod = mat_tot.transpose() * pui
prod = prod.transpose()

NameError: name 'sites' is not defined

In [ ]:
s = CyClpSimplex()
y = s.addVariable('y', 642) # variable bin qui vaut 1 qd on prend le site et 0 sinon

#La matrice A est la matrice composee des 1 et 0 des site offshore
A = cap_off

#La matrice B est composee des puissance de chaque site
B = pui

#La matrice R est la matrice de rendement de chaque periode (colone) pour chaque site (ligne)
R = mat_tot

#La matrice X est la matrice des production d'energie de chaque (colone) de chaque site (ligne)
X = prod

# ajout des contraintes
s += A*y <= k # proportion offshore
s += B*y == P # puissance totale
s += 0 # variablilte

#Set the objective function
s.objective = -(R*X*Y) # car simplexe minimize tjr

# Solve using primal Simplex
s.primal()
print(s.primalVariableSolution['x'])
